In [1]:
import os
import gc
import time

import math
import datetime
from math import log, floor
from sklearn.neighbors import KDTree

import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.utils import shuffle
from tqdm.notebook import tqdm as tqdm

import seaborn as sns
from matplotlib import colors
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import pywt
from statsmodels.robust import mad


import scipy
import statsmodels
from scipy import signal
import statsmodels.api as sm
from fbprophet import Prophet
from scipy.signal import butter, deconvolve
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, StackingRegressor, GradientBoostingRegressor
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

from typing import Union

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm_notebook as tqdm
import os
import gc
import warnings

import pandas as pd
from pandas.plotting import register_matplotlib_converters
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

from multiprocessing import cpu_count
from joblib import Parallel, parallel_backend
from joblib import delayed
import multiprocessing

In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics: 
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


def read_data():
    sell_prices_df = pd.read_csv(INPUT_DIR_PATH + 'sell_prices.csv')
    sell_prices_df = reduce_mem_usage(sell_prices_df)
    print('Sell prices has {} rows and {} columns'.format(sell_prices_df.shape[0], sell_prices_df.shape[1]))

    calendar_df = pd.read_csv(INPUT_DIR_PATH + 'calendar.csv')
    calendar_df = reduce_mem_usage(calendar_df)
    print('Calendar has {} rows and {} columns'.format(calendar_df.shape[0], calendar_df.shape[1]))

    sales_train_validation_df = pd.read_csv(INPUT_DIR_PATH + 'sales_train_validation.csv')
    print('Sales train validation has {} rows and {} columns'.format(sales_train_validation_df.shape[0], sales_train_validation_df.shape[1]))

    submission_df = pd.read_csv(INPUT_DIR_PATH + 'sample_submission.csv')
    return sell_prices_df, calendar_df, sales_train_validation_df, submission_df

In [3]:
INPUT_DIR_PATH = '../input/m5-forecasting-accuracy/'
prices, calendar, sales_train, submission_df = read_data()

Mem. usage decreased to 130.48 Mb (37.5% reduction)
Sell prices has 6841121 rows and 4 columns
Mem. usage decreased to  0.12 Mb (41.9% reduction)
Calendar has 1969 rows and 14 columns
Sales train validation has 30490 rows and 1919 columns


In [4]:
dataset = sales_train

del sales_train
gc.collect()

20

In [5]:
import pickle


with open("../input/1-2-stats-id/2_Z_CA.pkl", "rb") as z2:
    Z_2 = z2.read()
        
with open("../input/1-2-stats-id/2_window_CA.pkl", "rb") as window2:
    window_2 = window2.read()
    
with open("../input/1-2-stats-id/2_list_cal_CA.pkl", "rb") as calendar_2:
    calendar_2 = calendar_2.read()

Z_3 = pickle.loads(Z_2)

window_3 = pickle.loads(window_2)

calendar_3 = pickle.loads(calendar_2)



In [6]:
from scipy.ndimage.interpolation import shift
from scipy.stats import gmean
from scipy.stats import skew
from scipy.stats import kurtosis

# def update_statistics(data, n_in_0=1,
#                        n_in_1=1,
#                        n_in_2=1,
#                        n_in_3=1,
#                        n_in_4=1,dropnan=True):
    
# #     n_vars = 1 if type(data) is list else data.shape[1]
#     cols_pred, cols_0, cols_1, cols_2, cols_3, cols_4= list(),list(),list(),list(),list(),list()
    
#     for i in range(0, n_out):
#         shif_out = shift(data, -i, mode='constant', cval=-2000).astype(np.float16)
#         shif_out[shif_out ==-2000] = np.nan
#         cols_pred.append(shif_out)    
#     for i in range(28 + n_in_0, 28, -1):
#         shif_0 = shift(data, i, mode='constant', cval=-2000).astype(np.float16)
#         shif_0 [shif_0 ==-2000] = np.nan
#         cols_0.append(shif_0)        
#     for i in range(28 + n_in_1, 28,-1):
#         shif_1 = shift(data, i, mode='constant', cval=-2000).astype(np.float16)
#         shif_1 [shif_1 ==-2000] = np.nan
#         cols_1.append(shif_1)        
#     for i in range(28 + n_in_2, 28, -1):
#         shif_2 = shift(data, i, mode='constant', cval=-2000).astype(np.float16)
#         shif_2 [shif_2 ==-2000] = np.nan
#         cols_2.append(shif_2)        
#     for i in range(28 + n_in_3, 28, -1):
#         shif_3 = shift(data, i, mode='constant', cval=-2000).astype(np.float16)
#         shif_3 [shif_3 ==-2000] = np.nan
#         cols_3.append(shif_3)    
#     for i in range(28 + n_in_4, 28, -1):
#         shif_4 = shift(data, i, mode='constant', cval=-2000).astype(np.float32)
#         shif_4 [shif_4 ==-2000] = np.nan
#         cols_4.append(shif_4)        
     
   
#     # put it all together
#     agg_out_pred = np.transpose(np.vstack(cols_pred))
#     agg_out_0 = np.transpose(np.vstack(cols_0))    
#     agg_out_1 = np.transpose(np.vstack(cols_1))    
#     agg_out_2 = np.transpose(np.vstack(cols_2))    
#     agg_out_3 = np.transpose(np.vstack(cols_3))    
#     agg_out_4 = np.transpose(np.vstack(cols_4))
    
#     med_0 = np.mean(agg_out_0,axis=1).reshape(-1,1)
#     std_0 = np.std(agg_out_0,axis=1).reshape(-1,1)
    
#     med_1 = np.mean(agg_out_1,axis=1).reshape(-1,1)
#     std_1 = np.std(agg_out_1,axis=1).reshape(-1,1)
    
#     med_2 = np.mean(agg_out_2,axis=1).reshape(-1,1)
#     std_2 = np.std(agg_out_2,axis=1).reshape(-1,1)
    
#     med_3 = np.mean(agg_out_3,axis=1).reshape(-1,1)
#     std_3 = np.std(agg_out_3,axis=1).reshape(-1,1)
    
#     med_4 = np.mean(agg_out_4,axis=1).reshape(-1,1)
#     std_4 = np.std(agg_out_4,axis=1).reshape(-1,1)


#     statistics = np.hstack((std_4, std_3, std_2, std_1,std_0,
#                     med_4, med_3, med_2, med_1,med_0))

#     updated_stat = statistics[-1,:]#.reshape(1,-1)
   
#     return updated_stat

In [7]:
def update_calendar_statistics(data,
                               n_lags = 1,
                               n_in_0=1,
                               n_in_1=1,
                               n_in_2=1,
                               n_in_3=1,n_in_4=1,dropnan=True):

    
    XY = np.zeros((data.shape[0],1))
    
    DAY = 27
    
    LAGGING = n_lags
    cols = list() 
    for i in range(LAGGING+DAY, DAY, -1):
        shif = shift(data, i, mode='constant', cval=-2000).astype(np.float16)
        shif[shif ==-2000] = np.nan
        cols.append(shif)
    LAGGING_DAY = np.transpose(np.vstack(cols)) 
    del cols
    XY = np.hstack((LAGGING_DAY, XY))

    lags = [n_in_0, n_in_1, n_in_2, n_in_3, n_in_4]
    for lag in lags:
        cols_0 = list() 
        for i in range( lag+DAY, DAY, -1):
            shif_0 = shift(data, i, mode='constant', cval=-2000).astype(np.float32)
            shif_0[shif_0 ==-2000] = np.nan
            cols_0.append(shif_0)
        LAG = np.transpose(np.vstack(cols_0)) 
        med_0 = np.mean(LAG, axis=1).reshape(-1,1)
        std_0 = np.std(LAG, axis=1).reshape(-1,1)
        del cols_0, LAG
        XY = np.hstack((std_0, med_0, XY))

    shifts = [0, 6, 13]
    for day in shifts:
        lags = [n_in_0, n_in_1, n_in_2, n_in_3]
        for lag in lags:
            cols = list() 
            for i in range(lag +day, day, -1):
                shif = shift(data, i, mode='constant', cval=-2000).astype(np.float32)
                shif[shif ==-2000] = np.nan
                cols.append(shif)
            LAG_SHIFT = np.transpose(np.vstack(cols)) 
            med_0 = np.mean(LAG_SHIFT, axis=1).reshape(-1,1)
            del cols, LAG_SHIFT
            XY = np.hstack((med_0, XY))
            
    return XY[-1,:-1]

In [8]:
# make a recursive multi-step forecast
def forecast_recursive(model, Z, Window_Frame, list_ID_PR_CAL, lag, mean_0, mean_1,mean_2, mean_3, mean_4):
    
    # PREDICTIONS FRAME 
    FRAME_PRED = np.zeros((Z.shape[0],1))
    # 1st CYCLE: Recursive Predictions
    for j in range(28):
        # Initialisation
        if j == 0:
            print(Z.shape)
            PRED = model.predict(np.delete(Z,[3,4],1)).reshape(-1,1)
            NEW_WINDOW = np.hstack((Window_Frame, PRED))
            FRAME_PRED = np.hstack((FRAME_PRED, PRED))
            print(FRAME_PRED[:,1:])
        # Recursive Predictions   
        else:
            t=0
            list_row_z = []
            for row in NEW_WINDOW:
                stats_updated = update_calendar_statistics(row,
                                                           lag,
                                                           mean_0, mean_1, mean_2, mean_3, mean_4).reshape(1,-1) 
                id_pr_cal = list_ID_PR_CAL[t][j-1,:].reshape(1,-1)
                id_pr_cal = np.delete(id_pr_cal,[3,4],1)
                row_z = np.hstack((id_pr_cal,stats_updated))
                list_row_z.append(row_z )
                t=t+1
            Z_new = np.vstack(list_row_z)
            PRED = model.predict(Z_new).reshape(-1,1)
            NEW_WINDOW = np.hstack((NEW_WINDOW, PRED))
            FRAME_PRED = np.hstack((FRAME_PRED, PRED))
            print(FRAME_PRED[:,1:])

    return FRAME_PRED[:,1:]

In [9]:
# make a recursive multi-step forecast
def forecast_direct(model, Z, Window_Frame, list_ID_PR_CAL, lag,  mean_0, mean_1,mean_2, mean_3, mean_4):
    # PREDICTIONS FRAME 
    FRAME_PRED = np.zeros((Z.shape[0],1))
    # 1st CYCLE: Recursive Predictions
    for j in range(28):
        # Initialisation
        if j == 0:
            print(Z.shape)
            PRED = model.predict(np.delete(Z,[3,4],1)).reshape(-1,1)
            FRAME_PRED = np.hstack((FRAME_PRED, PRED))
            print(FRAME_PRED[:,1:])
        # Direct Predictions   
        else:
            t=0
            list_row_z = []
            for row in Z:
                statistics = row[27:].reshape(1,-1) 
                id_pr_cal = list_ID_PR_CAL[t][j-1,:].reshape(1,-1)
                id_pr_cal = np.delete(id_pr_cal,[3,4],1)
                row_z = np.hstack((id_pr_cal,statistics))
                list_row_z.append(row_z )
                t=t+1
            Z_new = np.vstack(list_row_z)
            print(Z_new.shape)
            PRED = model.predict(Z_new).reshape(-1,1)
            FRAME_PRED = np.hstack((FRAME_PRED, PRED))
            print(FRAME_PRED[:,1:])
            print('')
            print(id_pr_cal)
            print('')
            print(t)
   
    return FRAME_PRED[:,1:]

In [10]:
n_out = 1
n_lags = 15
n_0 = 7
n_1 = 14
n_2 = 30
n_3 = 60
n_4 = 180

w1 = window_3[:3049,:]
w2 = window_3[3049:6098,:]

window = [w1,w2]

STORES = np.unique(Z_3[:,3])
print(STORES)
list_stores=[]
for store in STORES:
    print('')
    print('Store')
    print(store)
    print('')
    zi_store = Z_3[Z_3[:,3]==store]
    model = lgb.Booster(model_file='../input/model-train/model_ca_'+str(int(store)+2)+'.txt')
    store_forecasts = forecast_recursive(model, zi_store, window[int(store)], calendar_3,n_lags, n_0,
                                         n_1, n_2, n_3,n_4)
    list_stores.append(store_forecasts)
    
df = np.vstack(list_stores)
forecast = pd.DataFrame(df)
forecast.to_csv('forecast_CA_2.csv', index=False)

[0. 1.]

Store
0.0

(3049, 64)
[[1.04941347]
 [0.10414669]
 [0.27004363]
 ...
 [1.76248106]
 [0.18319685]
 [4.20607425]]
[[1.04941347 0.94301659]
 [0.10414669 0.15786389]
 [0.27004363 0.2568005 ]
 ...
 [1.76248106 1.57350293]
 [0.18319685 0.51393715]
 [4.20607425 3.81890254]]
[[1.04941347 0.94301659 0.92703915]
 [0.10414669 0.15786389 0.14631029]
 [0.27004363 0.2568005  0.2384077 ]
 ...
 [1.76248106 1.57350293 1.54986387]
 [0.18319685 0.51393715 0.445885  ]
 [4.20607425 3.81890254 4.09205399]]
[[1.04941347 0.94301659 0.92703915 0.91315962]
 [0.10414669 0.15786389 0.14631029 0.14789997]
 [0.27004363 0.2568005  0.2384077  0.26316638]
 ...
 [1.76248106 1.57350293 1.54986387 1.44831733]
 [0.18319685 0.51393715 0.445885   0.43696753]
 [4.20607425 3.81890254 4.09205399 3.98183866]]
[[1.04941347 0.94301659 0.92703915 0.91315962 0.94350847]
 [0.10414669 0.15786389 0.14631029 0.14789997 0.16350825]
 [0.27004363 0.2568005  0.2384077  0.26316638 0.25941762]
 ...
 [1.76248106 1.57350293 1.54986387

In [11]:
# df = forecast_reverse(model, Z_3, window_3, calendar_3, n_0, n_1, n_2, n_3, n_4, n_5, n_6)
# forecast = pd.DataFrame(df)
# forecast.to_csv('forecast_CA.csv', index=False)

In [12]:
# df_forecast = pd.read_csv("../input/1-pred-ca/forecast_diff_CA.csv")

In [13]:
# forecast = pd.DataFrame([[-0.38658618,  0.03515502, -0.36224897],
#                          [0.18962422,  1.53729267, -0.16193974],
#                          [-0.60323886 ,-0.02146203, -0.11368382]])

# df_window = pd.DataFrame(window_sqrt_diff)
# transf = pd.concat([df_window, forecast],axis=1).fillna(0)
# v0 = v0.reshape(-1,1)

# transf

In [14]:
# pd.DataFrame(np.square(np.hstack((v0, transf)).cumsum(axis=1)))

In [15]:
# v0 = window_dataset[:,0]
# diff_window_df = pd.DataFrame(window_dataset).diff(axis=1)
# df_forecast_diff = pd.concat([diff_window_df, forecast],axis=1)
# df_forecast_diff
# df_v0 = df_forecast_diff.expanding(0,axis=1).sum()
# # for t in range (len(df_v0)):
# #     df_v0.iloc[t,:] = df_v0.iloc[t,:] + v0[t]
    
# df_v0.iloc[:,:]#.to_csv('forecast_CA.csv', index=False)